In [ ]:
library(dplyr)

In [ ]:
df <- read.csv("TestAllocate.csv", header = TRUE, sep = ",")
str(df)

In [ ]:
head(df)

In [ ]:
# # Remove extraneous variables 

# df$X <- NULL
# df$X.1 <- NULL
# df$X.2 <- NULL
# head(df, 3)

### <font color = lime> Convert the df char data types to numeric

In [ ]:
# Convert Excel dates when in the 1900 date format
df$date <- as.Date(df$date, origin = "1899-12-30")
df$date_s <- as.Date(df$date_s, origin = "1899-12-30")
head(df, 3)

In [ ]:
# create a test loop for moving through the df

# Define the target sum
target_sum <- 1000

# Initialize a variable to hold the cumulative sum
cumulative_sum <- 0

# Initialize a variable to hold the index where target sum is achieved
index <- 0

# Iterate over the column
for(i in 1:nrow(df)) {
  cumulative_sum <- cumulative_sum + df[i, 'quan']
    print(cumulative_sum)
  if(cumulative_sum == target_sum) {
      temp <- df[i, 'tally_q']
      df[i, 'tally_q'] <- temp
      }
  else if(cumulative_sum >= target_sum) {
    index <- i
    break
  }
}

# Print the index
print(index)
cumulative_sum

In [ ]:
index

In [ ]:
# Assuming 'df' is your dataframe and 'Quantity' is your column
df <- data.frame(Quantity = c(1, 2, 3, 4, 5), Other = c('a', 'b', 'c', 'd', 'e'))
test_values <- c(8, 6)
df
# Iterate over the test values
for(test_value in test_values) {
  # Initialize a variable to hold the cumulative sum
  cumulative_sum <- 0

  # Iterate over the rows
  for(i in 1:nrow(df)) {
    cumulative_sum <- cumulative_sum + df[i, 'Quantity']
    
    # Check if the cumulative sum exceeds the test value
    if(cumulative_sum > test_value) {
      # Calculate the excess and the adjusted quantity
      excess <- cumulative_sum - test_value
      adjusted_quantity <- df[i, 'Quantity'] - excess
      
      # Update the quantity in the current row
      df[i, 'Quantity'] <- adjusted_quantity
      
      # Create a new row with the excess quantity
      new_row <- df[i, ]
      new_row['Quantity'] <- excess
      
      # Insert the new row into the dataframe immediately below the current row
      df <- df %>% 
        add_row(.before = i+1, !!!new_row)
      
      # Break the loop
      break
    }
  }
}

# Print the updated dataframe
print(df)


## <font color = lime> Keeper!  It works

In [ ]:
df <- read.csv("TestAllocate.csv", header = TRUE, sep = ",")
df$X <- NULL
df$X.1 <- NULL
df$X.2 <- NULL



ct <<- 1
test_value <- c(850)
rat1 <<- 0

# function to allocate values within a df
inter <- function(df, test_value, ct) {
     cumulative_sum <- 0
      
# Iterate over the rows
  for(i in 1:nrow(df)) {
      
    cumulative_sum <- cumulative_sum + df[i, 'quan']
    
    # Check if the cumulative sum exceeds the test value
    if(cumulative_sum == test_value) {
        print("match amount")       
        }
      
    if(cumulative_sum >= test_value) {        
           
      # Calculate the excess and the adjusted quantity
      excess <- cumulative_sum - test_value         # 900 - 700 = 200
      adjusted_quantity <- df[i, 'quan'] - excess   # deduct 200 from existing value 
       
     # establish the ratio - need for row 1 and row 2   
     denom <- df[i, 'quan']    # 400    
     numer <- adjusted_quantity
      
      # Update the quantity in the current row
      df[i, 'quan'] <- adjusted_quantity            # assign new value to quan
        
      # Create a new row with the excess quantity (Do it here and not below the changes)
      new_row <- df[i, ]
      new_row['quan'] <- excess 
      
    
    # update the tally_q (first row)
      q1 <- df[i, 'tally_q'] - adjusted_quantity
      df[i, 'tally_q'] <- q1
        
    
   # first, establish a ratio to apply to variables in row 1, then allocate the variables
       
    # create the ratio
    rat1 <<- numer / denom
               
    # apply ratio to variables in row 1
    # cost var
    c1 <- df[i, 'cost'] * rat1
    df[i, 'cost'] <- c1  
    #tally_c var
    t_c <- df[i-1,'tally_c'] + c1
    df[i, 'tally_c'] <- t_c
    #proceeds var
    p1 <- df[i, 'proceeds'] * rat1
    df[i, 'proceeds'] <- p1
    # t_q2 var
    t_q2 <- df[i-1, 'tally_q2'] - adjusted_quantity
    df[i, 'tally_q2'] <- t_q2
    # t_c2 var
    t_c2 <- df[i-1, 'tally_c2'] - c1
    df[i, 'tally_c2'] <- t_c2
        
        
    # Insert the new row into the dataframe immediately below the current row
    df <- df %>% 
    add_row(.before = i+1, !!!new_row)
    ct <<- i+1        

#  PART 2
        
   # update the values in new row 2 
        
    # cost var
    c2 <- df[ct, 'cost'] * rat1
    df[ct, 'cost'] <- c2  
    #tally_c var
    t_c2 <- df[ct-1,'tally_c'] + c2
    df[ct, 'tally_c'] <- t_c2
    #proceeds var
    p2 <- df[ct, 'proceeds'] * rat1
    df[ct, 'proceeds'] <- p2
      # t_q2 var
    t_q22 <- df[ct-1, 'tally_q2'] - adjusted_quantity
    df[ct, 'tally_q2'] <- t_q22
    # t_c2 var
    t_c22 <- df[i-1, 'tally_c2'] - c2
    df[i, 'tally_c2'] <- t_c22      

  # Break the loop and move to the next test value
    ct <- ct + 1
  break
    
    }
    
  }
     return(df)
}


print(ct)
# make fct call
newdf <- inter(df, test_value, ct) 
tail(newdf, 3)


# Remove the unwanted row of NAs
newdf <- newdf[complete.cases(newdf), ]  
tail(newdf, 3)



# show the returned ct value


In [ ]:

ct <<- 1

sales <- c(1000,1000,1000,100,900,3000,6000,1000,1850,3000,3000,
           500,2000,3000,500,2000,810,1000,190,3000,1000,1100)


for(i in 1:length(sales)) {
    mod_df <- inter(df, sales[i], ct)
    print(mod_df)
    ct <<- ct + 1
    df <- df[complete.cases(df), ]
    }    
ct

head(mod_df,15)

In [ ]:
sales <- c(1000,1000,1000,100,900,3000,6000,1000,1850,3000,
           3000,500,2000,3000,500,2000,810,1000,190,3000,1000,1100)

ct <- 0

for(i in 1:length(sales)) {
    #tally sales
    ct <<- ct + sales[i]
    }    
ct



In [ ]:
# Remove extraneous variables 
df$X <- NULL
df$X.1 <- NULL
df$X.2 <- NULL
head(df, 3)

In [ ]:
df <- read.csv("TestAllocate.csv", header = TRUE, sep = ",")
df$X <- NULL
df$X.1 <- NULL
df$X.2 <- NULL

sales <- c(1000,1000,1000,100,900,3000,6000,1000,1850,3000,3000,500,2000,3000,500,2000,810,1000,190,3000,1000,1100)
length(sales)
str(sales)

start_row <- 1

for(i in start_row:nrow(df)) {
    print(start_row)
    # str(i)
    
    if(df[i, 'quan'] <= sales[i])  {
        temp <- df[i, 'quan'] 
        df[i,'quan'] <- temp
    } else {
    ct <<- ct + sales[i]
        }
    start_row <- start_row + 1
}
 
 
#tally
ct

In [ ]:


sales <- c(1000,1000,1000,100,900,3000,6000,1000,1850,3000,3000,500,2000,3000,500,2000,810,1000,190,3000,1000,1100)
length(sales)
# str(sales)
tally <- 0
start_row <- 1

for(i in 1:length(df))  { #   start_row:nrow(df)) {
    # cat("startrow is", start_row, "\n")
    tally <- tally + 1 
    # print(paste0(i,"helo"))
    # print(df[i, 'quan'])
    #print(start_row)
    
    temp <- df[i, 'quan'] 
    print(paste0("value of temp is ", temp))
    print(sales[i])
#   if(cumulative_sum >= test_value) {         
    if(temp <= sales[i])  {
        #temp <- df[i, 'quan'] 
        df[i,'quan'] <- temp
    } 
    else {
    ct <<- ct + sales[i]
        }
    start_row <- start_row + 1
}
 
 
print(paste0("The tally is :", tally))
ct

In [ ]:
df[22:23,]
head(df, 3)

In [ ]:
is.na(df[i, 'quan'])        
is.na(df[1, 'quan'])
   

In [ ]:
df$X <- NULL
df$X.1 <- NULL
df$X.2 <- NULL

In [ ]:
df$quan <- as.integer(df$quan)

In [ ]:
head(df)

In [ ]:
df <- read.csv("TestAllocate.csv", header = TRUE, sep = ",")

cumulative_sum <- 0

start_row <- 1

#Iterate over the sales values
for(value in sales) {
    # Iterate over the rows
    for(i in start_row:nrow(df)) {
        cumulative_sum <- 0
        cumulative_sum <- cumulative_sum + df[i, 'quan']
        # check if the cumulative sum equals the test value in sales[i], if so, do nada
        if(cumulative_sum <= sales[i]) {
            temp <- df[i, 'quan']
            df[i, 'quan'] <- temp                 
        }
        # Check if the cumulative sum exceeds the test value
        else if(cumulative_sum >= sales[i]) {        
    
            # Calculate the excess and the adjusted quantity
            excess <- cumulative_sum - test_value         # 900 - 700 = 200
            adjusted_quantity <- df[i, 'quan'] - excess   # deduct 200 from existing value 

            # establish the ratio - need for row 1 and row 2   
            denom <- df[i, 'quan']    # 400    
            numer <- adjusted_quantity

            # Update the quantity in the current row
            df[i, 'quan'] <- adjusted_quantity            # assign new value to quan

            # Create a new row with the excess quantity (Do it here and not below the changes)
            new_row <- df[i, ]
            new_row['quan'] <- excess  

            start_row <- i + 1
            break
        }
      }
    }
df

In [ ]:
# Assuming 'vec' is your vector and 'df' is your dataframe
start_row <- 1
for(value in vec) {
  for(i in start_row:nrow(df)) {
    if(df[i, 'var'] == value) {  # Replace 'var' with your actual column name
      # Execute your code here
      print(paste("Match found for", value, "at row", i))
      start_row <- i + 1
      break
    }
  }
}


In [ ]:
length(df)

In [ ]:
stopper


## <font color = lime> Part 2
    
<div style="margin-left:40px;">
Allocate the other data in the matched row
</div> 


In [ ]:
df <- read.csv("locate.csv", header = TRUE, sep = ",")

ct <<- 0
test_value <- c(1000)
rat1 <<- 0

# function to allocate values within a df
inter <- function(df, test_value, ct) {
     cumulative_sum <- 0
      
# Iterate over the rows
  for(i in 1:nrow(df)) {
      
    cumulative_sum <- cumulative_sum + df[i, 'quan']
    
    # Check if the cumulative sum exceeds the test value
    if(cumulative_sum >= test_value) {
        
     
           
      # Calculate the excess and the adjusted quantity
      excess <- cumulative_sum - test_value         # 900 - 700 = 200
      adjusted_quantity <- df[i, 'quan'] - excess   # deduct 200 from existing value 
       
     # establish the ratio - need for row 1 and row 2   
     denom <- df[i, 'quan']    # 400    
     numer <- adjusted_quantity
      
      # Update the quantity in the current row
      df[i, 'quan'] <- adjusted_quantity            # assign new value to quan
        
      # Create a new row with the excess quantity (Do it here and not below the changes)
      new_row <- df[i, ]
      new_row['quan'] <- excess 
      
    
    # update the tally_q (first row)
      q1 <- df[i, 'tally_q'] - adjusted_quantity
      df[i, 'tally_q'] <- q1
        
    
   # first, establish a ratio to apply to variables in row 1, then allocate the variables
       
    # create the ratio
    rat1 <<- numer / denom
               
    # apply ratio to variables in row 1
    # cost var
    c1 <- df[i, 'cost'] * rat1
    df[i, 'cost'] <- c1  
    #tally_c var
    t_c <- df[i-1,'tally_c'] + c1
    df[i, 'tally_c'] <- t_c
    #proceeds var
    p1 <- df[i, 'proceeds'] * rat1
    df[i, 'proceeds'] <- p1
    # t_q2 var
    t_q2 <- df[i-1, 'tally_q2'] - adjusted_quantity
    df[i, 'tally_q2'] <- t_q2
    # t_c2 var
    t_c2 <- df[i-1, 'tally_c2'] - c1
    df[i, 'tally_c2'] <- t_c2
        
        
    # Insert the new row into the dataframe immediately below the current row
    df <- df %>% 
    add_row(.before = i+1, !!!new_row)
    ct <<- i+1        

#  PART 2
        
   # update the values in new row 2 
        
    # cost var
    c2 <- df[ct, 'cost'] * rat1
    df[ct, 'cost'] <- c2  
    #tally_c var
    t_c2 <- df[ct-1,'tally_c'] + c2
    df[ct, 'tally_c'] <- t_c2
    #proceeds var
    p2 <- df[ct, 'proceeds'] * rat1
    df[ct, 'proceeds'] <- p2
      # t_q2 var
    t_q22 <- df[ct-1, 'tally_q2'] - adjusted_quantity
    df[ct, 'tally_q2'] <- t_q22
    # t_c2 var
    t_c22 <- df[i-1, 'tally_c2'] - c2
    df[i, 'tally_c2'] <- t_c22      

  # Break the loop and move to the next test value
  break
    
    }
    
  }
     return(df)
}

# make fct call

mod_df <- inter(df, test_value, ct)

# show the returned ct value
ct
rat1

head(mod_df,10)



In [ ]:
f <- function(x) x + 1
str(f)
('--------------------------------------------')
result <- f(60)                                   # Current age
result
('--------------------------------------------')
str(result)

print(f)